<a href="https://colab.research.google.com/github/ovecats/Precio_Bitcoin_USD_2023/blob/main/BTC_USD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Precio diario de Bitcoin USD con volumen (2015-2023)***


#**Instalar librerías**

In [1]:
# Instalar librerias
!pip install pyspark
!pip install -q kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=db5e9a902cb2d76b73071f73f2480f8715b71b8dd9f10327dc17c0ea8b273c02
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


# **Importar librerías**

In [2]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import zipfile
import os
import pandas as pd

# **Creación de una sesión con Spark**

In [3]:
#Iniciar conexión de spark
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# **Conexión con kaggle para descargar los datos**
Para descargar datos desde Kaggle es necesario crear un token de acceso. Para esto debes seguir los siguientes pasos:

1.   Ingresa a tu perfil de kaggle
2.   Seleccionar *Account* y bajar hasta la opción *API*
3.   Dar click en *Create New API Token* y se descargará un archivo JSON con   tus credenciales
4.   Subir el archivo JSON a Colab Notebook


In [4]:
# Creamos carpeta oculta en ambiente de linux sobre colab
!mkdir ~/.kaggle

In [5]:
# Copiar el archivo JSON  a la carpeta oculta que creamos
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [6]:
# Cambiamos los permisos para permitir lectura de las credenciales
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
# Descargamos archivos indicando el usuario del propietario de los datos en kaggle y el nombre de dataset
! kaggle datasets download prkhrawsthi/bitcoin-usd-daily-price-with-volume-2015-2023 --force

  0% 0.00/88.3k [00:00<?, ?B/s]
100% 88.3k/88.3k [00:00<00:00, 67.0MB/s]


In [8]:
# Descomprimir el archivo
for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

# **Lectura de datos**

## Lectura de datos con Pandas

In [9]:
df_pandas =pd.read_csv('/content/BTC-USD.csv')

## Lectura datos con PySpark

In [10]:
df = spark.read.csv('/content/BTC-USD.csv', header=True)

In [15]:
# Guardar el dataset en Drive
!cp BTC-USD.csv> "/content/drive/MyDrive/(datasets)"

/bin/bash: /content/drive/MyDrive/(datasets): No such file or directory


# **Procesamiento de datos**

## Número de filas y columnas

pandas:

```
df.shape
```



In [16]:
print((df.count(), len(df.columns)))

(3104, 7)


## Ver una muestra de los datos

pandas:

```
df.head()
```

In [17]:
df.show(5)

+----------+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close| Adj Close|  Volume|
+----------+----------+----------+----------+----------+----------+--------+
|2015-01-03|314.846008|315.149994|281.082001|281.082001|281.082001|33054400|
|2015-01-04|281.145996|287.230011|257.612000|264.195007|264.195007|55629100|
|2015-01-05|265.084015|278.341003|265.084015|274.473999|274.473999|43962800|
|2015-01-06|274.610992|287.553009|272.696014|286.188995|286.188995|23245700|
|2015-01-07|286.076996|298.753998|283.079010|294.337006|294.337006|24866800|
+----------+----------+----------+----------+----------+----------+--------+
only showing top 5 rows



## Esquema de los datos

pandas:

```
df.info()
```

In [18]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)



## Estadística descriptiva

pandas:

```
df.describe()
```

In [19]:
df.describe().show()

+-------+----------+------------------+------------------+------------------+------------------+------------------+--------------------+
|summary|      Date|              Open|              High|               Low|             Close|         Adj Close|              Volume|
+-------+----------+------------------+------------------+------------------+------------------+------------------+--------------------+
|  count|      3104|              3104|              3104|              3104|              3104|              3104|                3104|
|   mean|      null|14041.039379181955|14385.041531778288|13664.743745447107|14049.414853644279|14049.414853644279|1.715624286159181...|
| stddev|      null|16095.940890798087| 16500.08755236381| 15634.14027342463| 16093.55742594234| 16093.55742594234|1.962263477785498...|
|    min|2015-01-03|      10024.115234|       1002.099976|      10000.708008|      10005.000000|      10005.000000|         10014858959|
|    max|2023-07-03|        998.885010|  

## Filtrar el dataset

pandas:

```
df[df['Date']=='2023-07-03']
```

In [20]:
df.filter(df.Date == "2023-07-03").show()

+----------+------------+------------+------------+------------+------------+-----------+
|      Date|        Open|        High|         Low|       Close|   Adj Close|     Volume|
+----------+------------+------------+------------+------------+------------+-----------+
|2023-07-03|30616.166016|30789.113281|30602.289063|30652.357422|30652.357422|11762191360|
+----------+------------+------------+------------+------------+------------+-----------+



## Ordenar el dataset
pandas:

```
df.sort_values(by=['UTCDate'], ascending=False)
```

In [21]:
df.sort(F.col("Date").desc()).show(5)

+----------+------------+------------+------------+------------+------------+-----------+
|      Date|        Open|        High|         Low|       Close|   Adj Close|     Volume|
+----------+------------+------------+------------+------------+------------+-----------+
|2023-07-03|30616.166016|30789.113281|30602.289063|30652.357422|30652.357422|11762191360|
|2023-07-02|30587.269531|30766.140625|30264.019531|30620.769531|30620.769531|10533418042|
|2023-07-01|30471.847656|30641.289063|30328.865234|30590.078125|30590.078125| 9086606733|
|2023-06-30|30441.353516|31256.863281|29600.275391|30477.251953|30477.251953|26387306197|
|2023-06-29|30086.187500|30796.250000|30057.203125|30445.351563|30445.351563|13180860821|
+----------+------------+------------+------------+------------+------------+-----------+
only showing top 5 rows



## Eliminar una columna:

pandas:

```
df.drop(columns=['Adj Close'], inplace=True)
```

In [22]:
df = df.drop("Adj Close")
df.show(4)

+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close|  Volume|
+----------+----------+----------+----------+----------+--------+
|2015-01-03|314.846008|315.149994|281.082001|281.082001|33054400|
|2015-01-04|281.145996|287.230011|257.612000|264.195007|55629100|
|2015-01-05|265.084015|278.341003|265.084015|274.473999|43962800|
|2015-01-06|274.610992|287.553009|272.696014|286.188995|23245700|
+----------+----------+----------+----------+----------+--------+
only showing top 4 rows



## Eliminar datos nulos:

pandas:

```
df.dropna(inplace=True)
```

In [23]:
df = df.na.drop()
df.show(4)

+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close|  Volume|
+----------+----------+----------+----------+----------+--------+
|2015-01-03|314.846008|315.149994|281.082001|281.082001|33054400|
|2015-01-04|281.145996|287.230011|257.612000|264.195007|55629100|
|2015-01-05|265.084015|278.341003|265.084015|274.473999|43962800|
|2015-01-06|274.610992|287.553009|272.696014|286.188995|23245700|
+----------+----------+----------+----------+----------+--------+
only showing top 4 rows



In [24]:
print((df.count(), len(df.columns)))

(3104, 6)


## Convertir datos a DataFrame de Pandas:

In [25]:
df_Pandas = df.select("Date", "Open", "High").limit(50).toPandas()

In [26]:
df_Pandas.head()

,Date,Open,High
0,2015-01-03,314.846008,315.149994
1,2015-01-04,281.145996,287.230011
2,2015-01-05,265.084015,278.341003
3,2015-01-06,274.610992,287.553009
4,2015-01-07,286.076996,298.753998


In [27]:
df_Pandas.shape

(50, 3)

## UDF

Los UDF de Pyspark son similares a los métodos .map () y .apply () de pandas 'series y dataframes'.

Eliminar guion en un string y reemplazar por slash:

In [28]:
UDF_python = F.udf(lambda s: s.replace("-", "/"), StringType())
df = df.withColumn("Date", UDF_python("Date"))

In [29]:
df.show(5)

+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close|  Volume|
+----------+----------+----------+----------+----------+--------+
|2015/01/03|314.846008|315.149994|281.082001|281.082001|33054400|
|2015/01/04|281.145996|287.230011|257.612000|264.195007|55629100|
|2015/01/05|265.084015|278.341003|265.084015|274.473999|43962800|
|2015/01/06|274.610992|287.553009|272.696014|286.188995|23245700|
|2015/01/07|286.076996|298.753998|283.079010|294.337006|24866800|
+----------+----------+----------+----------+----------+--------+
only showing top 5 rows



## Agrupar datos

pandas:

```
df.groupby('Event').count()
```

In [33]:
from pyspark.sql.functions import year
from pyspark.sql.functions import date_format, to_date
from pyspark.sql.functions import substring

In [34]:
# Agrupa por año utilizando substring() para extraer los primeros 4 caracteres
df = df.withColumn("Year", substring(df["Date"], 1, 4))

# Agrupa por año y realiza el recuento de filas por año
result = df.groupBy("Year").count()

# Muestra los resultados
result.show()

+----+-----+
|Year|count|
+----+-----+
|2016|  366|
|2020|  366|
|2019|  365|
|2017|  365|
|2018|  365|
|2022|  365|
|2023|  184|
|2021|  365|
|2015|  363|
+----+-----+



## Cambiar tipo de dato de una columna

pandas:

```
df['Date] = pd.to_datetime(df['UTCDate'], format="%Y/%m/%d")
```

In [35]:
# Convierte la columna "Date" al tipo de datos date utilizando to_date()
df = df.withColumn("Date", to_date(df["Date"], "yyyy-MM-dd"))

# Muestra el esquema del DataFrame para verificar el cambio de tipo de datos
df.printSchema()

# Muestra los primeros registros del DataFrame
df.show()

root
 |-- Date: date (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Year: string (nullable = true)

+----+----------+----------+----------+----------+--------+----+
|Date|      Open|      High|       Low|     Close|  Volume|Year|
+----+----------+----------+----------+----------+--------+----+
|null|314.846008|315.149994|281.082001|281.082001|33054400|2015|
|null|281.145996|287.230011|257.612000|264.195007|55629100|2015|
|null|265.084015|278.341003|265.084015|274.473999|43962800|2015|
|null|274.610992|287.553009|272.696014|286.188995|23245700|2015|
|null|286.076996|298.753998|283.079010|294.337006|24866800|2015|
|null|294.135010|294.135010|282.174988|283.348999|19982500|2015|
|null|282.382996|291.114014|280.532990|290.407990|18718600|2015|
|null|287.303009|288.127014|273.966003|274.795990|15264300|2015|
|null|274.608002|279.638000

## Renombrar una columna

pandas:

```
df.rename(columns={'Opening Type':'Opening}, inplace=True)
```

In [38]:
df = df.withColumnRenamed("Year", "Fecha")

In [39]:
df.show(3)

+----+----------+----------+----------+----------+--------+-----+
|Date|      Open|      High|       Low|     Close|  Volume|Fecha|
+----+----------+----------+----------+----------+--------+-----+
|null|314.846008|315.149994|281.082001|281.082001|33054400| 2015|
|null|281.145996|287.230011|257.612000|264.195007|55629100| 2015|
|null|265.084015|278.341003|265.084015|274.473999|43962800| 2015|
+----+----------+----------+----------+----------+--------+-----+
only showing top 3 rows



# **Spark SQL en Python**

## Creación de vista del dataframe

In [41]:
df.createOrReplaceTempView("btc_usd")

## Muestra de los datos

In [42]:
spark.sql("SELECT COUNT(*) FROM btc_usd").show()

+--------+
|count(1)|
+--------+
|    3104|
+--------+



## Valores únicos de una columna

In [44]:
spark.sql("SELECT DISTINCT Open FROM btc_usd").count()

3102

## Calculos de una columna

In [54]:
spark.sql("SELECT AVG(Open) as avg_apertura, AVG(High) as avg_alto  FROM btc_usd").show()

+------------------+------------------+
|      avg_apertura|          avg_alto|
+------------------+------------------+
|14041.039379181955|14385.041531778288|
+------------------+------------------+

